# lmports and definitions

In [1]:
# variable       delta_coherence_calculation 
# Class          Captial
# function       butterBandpassFilter
# meegkiit install -- command window
# variable names -- solved - next meeting
# definition of function -- solved - next meeting
# Visualization interpretation -- future
# welch - methods extraction -- future 
# Introduction to asymetry, coherence , ... by Pegah

# parameter      delta_coherence_calculation
# variable       var_delta_coherence_calculation 
# Class          Captial
# function       butterBandpassFilter
# internal variables or parameter name with _ at the beginning eg: _counter
import mne
import pandas as pd
import numpy as np
import scipy.io
import scipy.signal as ss
import pyedflib
from scipy.signal import hilbert,butter, lfilter,welch
from scipy.integrate import simps
from meegkit.asr import ASR
from meegkit.utils.matrix import sliding_window
from mne.time_frequency import psd_array_multitaper
from neurodsp.utils.download import load_ndsp_data
sampling_rate = 1000
#frequency rythms
#Delta:0.5-4
#Theta:4-8
#Alpha: 8-13
#Beta: 13-30
#Gamma:30-45
#eeg channels corresponding brain area
# left frontal (Fl)=fp1,f3,f7=0,2,10
# right frontal (Fr)=fp2,f4,f8=1,3,11
# left central (C)=t3, c3=12,4
# right central (C)=t4, c4=13,5
# left parietal-occipital (lPO)=t5,p3,o1=14,6,8
# right parietal-occipital (rPO)=t6,p4,o2=15,7,9
# midline (M)=fz,cz,pz=17,16,18

In [4]:
def butterBandpass(data, lower_limit_filter, upper_limit_filter, sampling_rate, order=4):
    """
    This func is for filtering signal between lower and upper bounds
    the methods are used from scipy.signal lib
    """
    nyquist_coeff = 0.5 * sampling_rate
    low_frequences_filter = lower_limit_filter / nyquist_coeff
    high_frequences_filter = upper_limit_filter / nyquist_coeff
    numerator_filter, denominator_filter = butter(order, 
                                                  [low_frequences_filter, high_frequences_filter],
                                                  btype='band')
    # based on numinator and denominator the filter signal ...                                            )
    filtered_signal = lfilter(numerator_filter, denominator_filter, data)
    return filtered_signal


In [31]:
def applyArtifactSubspaceReconstruction(raw, sfreq=sampling_rate, cutoff=2.5, 
                                        blocksize=100, win_len=0.5,
                                        win_overlap=0.66, max_dropout_fraction=0.1,
                                        min_clean_fraction=0.25, name='asrfilter', method='euclid',
                                        estimator='scm'):
    """
    Goal: this function removes artifact specially those related to EOG noises
    """
    h, w = raw.shape
    
    # t = int(raw.shape[1] / sfreq)
    # take asr ....
    asr = ASR(sfreq=sfreq, cutoff=cutoff, blocksize=blocksize, win_len=win_len,
        win_overlap=win_overlap, max_dropout_fraction=max_dropout_fraction,
        min_clean_fraction=min_clean_fraction, name=name, method=method,
        estimator=estimator)
        
    # method='euclid'
    # train_idx = np.arange(0 * sfreq, t * sfreq, dtype=int)
    
    # short description ....
    _, sample_mask = asr.fit(raw)
    
    # Apply filter using sliding (non-overlapping) windows
    # name X, Y
    # description
    
    #sliding_window function converts our 2-D data(number of channels by number of recorded samples) dataset to a 3-D function(number of channels by time in second by sampling rate) 
    X = sliding_window(raw, window=int(sfreq), step=int(sfreq))
    
    Y = np.zeros_like(X)
    for i in range(X.shape[1]):
        Y[:, i, :] = asr.transform(X[:, i, :])
    
   # reshape to (n_chans, n_times)
    clean = Y.reshape(h, -1)
    return clean

In [10]:
def bandPower(data, sampling_rate, frequency_band, method='welch', window_sec=None, relative=False):
    frequency_band = np.asarray(frequency_band)
    low_limitation, high_limitation = frequency_band
    # Compute the modified periodogram (Welch)
    if method == 'welch':
        if window_sec is not None:
            num_of_sample_per_segment = window_sec * sampling_rate
        else:
            num_of_sample_per_segment = (2 / low_limitation) * sampling_rate
        freqs, psd = welch(data, sampling_rate, nperseg=num_of_sample_per_segment)
    elif method == 'multitaper':
        psd, freqs = psd_array_multitaper(data, sampling_rate, adaptive=True,
                                          normalization='full', verbose=0)
    # Frequency resolution
    freq_res = freqs[1] - freqs[0]

    # Find index of band in frequency vector
    idx_band = np.logical_and(freqs >= low_limitation, freqs <= high_limitation)

    # Integral approximation of the spectrum using parabola (Simpson's rule)
    bp = simps(psd[idx_band], dx=freq_res)

    if relative:
        bp /= simps(psd, dx=freq_res)
    return bp

In [19]:
def myBandPower(data, sampling_rate, 
                frequency_band1,frequency_band2,frequency_band3,frequency_band4,frequency_band5,frequency_band6,
                frequency_band7,frequency_band8,frequency_band9,frequency_band10,
                method='welch', window_sec=None):
    frequency_band1 = np.asarray(frequency_band1)
    frequency_band2 = np.asarray(frequency_band2)
    frequency_band3 = np.asarray(frequency_band3)
    frequency_band4 = np.asarray(frequency_band4)
    frequency_band5 = np.asarray(frequency_band5)
    frequency_band6 = np.asarray(frequency_band6)
    frequency_band7 = np.asarray(frequency_band7)
    frequency_band8 = np.asarray(frequency_band8)
    frequency_band9 = np.asarray(frequency_band9)
    frequency_band10 = np.asarray(frequency_band10)
    
    low_limitation1, high_limitation1 = frequency_band1
    low_limitation2, high_limitation2 = frequency_band2
    low_limitation3, high_limitation3 = frequency_band3
    low_limitation4, high_limitation4 = frequency_band4
    low_limitation5, high_limitation5 = frequency_band5
    low_limitation6, high_limitation6 = frequency_band6
    low_limitation7, high_limitation7 = frequency_band7
    low_limitation8, high_limitation8 = frequency_band8
    low_limitation9, high_limitation9 = frequency_band9
    low_limitation10, high_limitation10 = frequency_band10
    # Compute the modified periodogram (Welch)
    if method == 'welch':
        if window_sec is not None:
            num_of_sample_per_segment = window_sec * sampling_rate
        else:
            num_of_sample_per_segment = (2 / low_limitation) * sampling_rate
        freqs, psd = welch(data, sampling_rate, nperseg=num_of_sample_per_segment)
    elif method == 'multitaper':
        psd, freqs = psd_array_multitaper(data, sampling_rate, adaptive=True,
                                          normalization='full', verbose=0)
    # Frequency resolution
    freq_res = freqs[1] - freqs[0]

    # Find index of band in frequency vector
    idx_band1 = np.logical_and(freqs >= low_limitation1, freqs <= high_limitation1)
    idx_band2 = np.logical_and(freqs >= low_limitation2, freqs <= high_limitation2)
    idx_band3 = np.logical_and(freqs >= low_limitation3, freqs <= high_limitation3)
    idx_band4 = np.logical_and(freqs >= low_limitation4, freqs <= high_limitation4)
    idx_band5 = np.logical_and(freqs >= low_limitation5, freqs <= high_limitation5)
    idx_band6 = np.logical_and(freqs >= low_limitation6, freqs <= high_limitation6)
    idx_band7 = np.logical_and(freqs >= low_limitation7, freqs <= high_limitation7)
    idx_band8 = np.logical_and(freqs >= low_limitation8, freqs <= high_limitation8)
    idx_band9 = np.logical_and(freqs >= low_limitation9, freqs <= high_limitation9)
    idx_band10 = np.logical_and(freqs >= low_limitation10, freqs <= high_limitation10)

    # Integral approximation of the spectrum using parabola (Simpson's rule)
    bp1 = simps(psd[idx_band1], dx=freq_res)
    bp2 = simps(psd[idx_band2], dx=freq_res)
    bp3 = simps(psd[idx_band3], dx=freq_res)
    bp4 = simps(psd[idx_band4], dx=freq_res)
    bp5 = simps(psd[idx_band5], dx=freq_res)
    bp6 = simps(psd[idx_band6], dx=freq_res)
    bp7 = simps(psd[idx_band7], dx=freq_res)
    bp8 = simps(psd[idx_band8], dx=freq_res)
    bp9 = simps(psd[idx_band9], dx=freq_res)
    bp10 = simps(psd[idx_band10], dx=freq_res)
    bp_general=simps(psd, dx=freq_res)
    
    bp_r1=bp1/bp_general
    bp_r2=bp2/bp_general
    bp_r3=bp3/bp_general
    bp_r4=bp4/bp_general
    bp_r5=bp5/bp_general
    bp_r6=bp6/bp_general
    bp_r7=bp7/bp_general
    bp_r8=bp8/bp_general
    bp_r9=bp9/bp_general
    bp_r10=bp10/bp_general
    bp=[bp1,bp2,bp3,bp4,bp5,bp6,bp7,bp8,bp9,bp10,bp_r1,bp_r2,bp_r3,bp_r4,bp_r5,bp_r6,bp_r7,bp_r8,bp_r9,bp_r10]       
    return bp

In [ ]:
def calcRms(x, scale):
    """
    windowed Root Mean Square (RMS) with linear detrending.
    
    Args:
    -----
      *x* : numpy.array
        one dimensional data vector
      *scale* : int
        length of the window in which RMS will be calculaed
    Returns:
    --------
      *rms* : numpy.array
        RMS data in each window with length len(x)//scale
    """
    # making an array with data divided in windows
    shape = (x.shape[0]//scale, scale)
    X = np.lib.stride_tricks.as_strided(x,shape=shape)
    # vector of x-axis points to regression
    scale_ax = np.arange(scale)
    rms = np.zeros(X.shape[0])
    for e, xcut in enumerate(X):
        coeff = np.polyfit(scale_ax, xcut, 1)
        xfit = np.polyval(coeff, scale_ax)
        # detrending and computing RMS of each window
        rms[e] = np.sqrt(np.mean((xcut-xfit)**2))
    return rms


In [ ]:
def dfa(x, scale_lim=[5,9], scale_dens=0.32, show=False):
    """
    Detrended Fluctuation Analysis - measures power law scaling coefficient
    of the given signal *x*.
    More details about the algorithm you can find e.g. here:
    Hardstone, R. et al. Detrended fluctuation analysis: A scale-free 
    view on neuronal oscillations, (2012).
    Args:
    -----
      *x* : numpy.array
        one dimensional data vector
      *scale_lim* = [5,9] : list of length 2 
        boundaries of the scale, where scale means windows among which RMS
        is calculated. Numbers from list are exponents of 2 to the power
        of X, eg. [5,9] is in fact [2**5, 2**9].
        You can think of it that if your signal is sampled with F_s = 128 Hz,
        then the lowest considered scale would be 2**5/128 = 32/128 = 0.25,
        so 250 ms.
        mine: fs=1000, 2**5/1000=32/1000=0.032
      *scale_dens* = 0.25 : float
        density of scale divisions, eg. for 0.25 we get 2**[5, 5.25, 5.5, ... ] 
      *show* = False
        if True it shows matplotlib log-log plot.
    Returns:
    --------
      *scales* : numpy.array
        vector of scales (x axis)
      *fluct* : numpy.array
        fluctuation function values (y axis)
      *alpha* : float
        estimation of DFA exponent
    """
    # cumulative sum of data with substracted offset
    y = np.cumsum(x - np.mean(x))
    scales = (2**np.arange(scale_lim[0], scale_lim[1], scale_dens)).astype(np.int)
    fluct = np.zeros(len(scales))
    # computing RMS for each window
    for e, sc in enumerate(scales):
        fluct[e] = np.sqrt(np.mean(calcRms(y, sc)**2))
    # fitting a line to rms data
    coeff = np.polyfit(np.log2(scales), np.log2(fluct), 1)
    if show:
        fluctfit = 2**np.polyval(coeff,np.log2(scales))
        plt.loglog(scales, fluct, 'bo')
        plt.loglog(scales, fluctfit, 'r', label=r'$\alpha$ = %0.2f'%coeff[0])
        plt.title('DFA')
        plt.xlabel(r'$\log_{10}$(time window)')
        plt.ylabel(r'$\log_{10}$<F(t)>')
        plt.legend()
        plt.show()
    return scales, fluct, coeff[0]

# load and preprocess

In [12]:
raw= mne.io.read_raw_eeglab('D:\WM_openneuro_dataset\sub-064\eeg\sub-064_task-rest_eeg.set')
all_events, all_event_id = mne.events_from_annotations(raw)
my_events= mne.pick_events(all_events)
epochs = mne.Epochs(raw,my_events, tmin=0.0, tmax=240.0, baseline=(0, 0))
data=epochs.to_data_frame()
data.drop('time', axis=1, inplace=True)
data.drop('condition', axis=1, inplace=True)
data.drop('epoch', axis=1, inplace=True)
data.drop([0],axis=0,inplace=True)
edata=data.to_numpy(dtype ='float32')
clm,rw =data.shape
eeg=np.reshape(edata,(rw,-1))

filter_signal = butterBandpass(eeg, 1, 45, 1000)
filter_asr_signal = applyArtifactSubspaceReconstruction(filter_signal)
np.save('s64_clean_resting_state', filter_asr_signal)

C:\ProgramData\Anaconda3\lib\site-packages\mne\externals\pymatreader\utils.py:118: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '
<ipython-input-12-8b2d80a5f0ab>:1: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  raw= mne.io.read_raw_eeglab('D:\WM_openneuro_dataset\sub-064\eeg\sub-064_task-rest_eeg.set')
<ipython-input-12-8b2d80a5f0ab>:1: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw= mne.io.read_raw_eeglab('D:\WM_openneuro_dataset\sub-064\eeg\sub-064_task-rest_eeg.set')
<ipython-input-12-8b2d80a5f0ab>:1: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw= mne.io.read_raw_eeglab('D:\WM_openneuro_dataset\sub-064\eeg\sub

Used Annotations descriptions: ['boundary', 'eyes opened']
Not setting metadata
Not setting metadata
2 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 2 events and 240001 original time points ...
1 bad epochs dropped


# absolout and relative band power

In [20]:
delta_p = delta_p_rel = theta_p=theta_p_rel=alpha_p=alpha_p_rel=beta_p=beta_p_rel=gamma_p= gamma_p_rel=np.zeros(21)
for ii in range (0,20):
    band_power_all_band[ii] = bandpower(filter_asr_signal[ii,:], sampling_rate, [1, 4],[4, 8],
                                        [8, 12],[12, 25], [25, 30],[30, 40],[40, 48],[8, 10],[10, 12],[12, 15],
                                        [15, 18], [18, 25], [30, 35], [35, 40],'multitaper')
    #delta,  theta, alpha,  beta,    h_beta,  gamma,   h_gamma, alpha_1,alpha_2, beta_1,  beta_2, beta_3,  gama_1,  gama_2
    #[1, 4],[4, 8],[8, 12],[12, 25],[25, 30],[30, 40],[40, 48],[8, 10],[10, 12],[12, 15],[15, 18],[18, 25],[30, 35],[35, 40]
    np.save('s64_band_power_all_band', band_power_all_band)
#ratio-power
#db = bandpower(filter_asr_signal[1,:], sampling_rate, [0.5, 4], 'multitaper') / bandpower(filter_asr_signal[1,:], sampling_rate, [12, 30], 'multitaper')

NameError: name 'filter_asr_signal' is not defined

# amplitude asymmetry

In [26]:
Amp_assym_delta=np.zeros((21,21))
Amp_assym_theta=np.zeros((21,21))
Amp_assym_alpha=np.zeros((21,21))
Amp_assym_beta=np.zeros((21,21))
Amp_assym_h_beta=np.zeros((21,21))
Amp_assym_beta1=np.zeros((21,21))
Amp_assym_beta2=np.zeros((21,21))
Amp_assym_beta3=np.zeros((21,21))

filter_signal_delta = butterBandpass(filter_asr_signal, 1, 4, 1000)
filter_signal_theta = butterBandpass(filter_asr_signal, 4, 8, 1000)
filter_signal_alpha = butterBandpass(filter_asr_signal, 8, 12, 1000)
filter_signal_beta = butterBandpass(filter_asr_signal, 12, 25, 1000)
filter_signal_h_beta = butterBandpass(filter_asr_signal, 25, 30, 1000)
filter_signal_beta1 = butterBandpass(filter_asr_signal, 12, 15, 1000)
filter_signal_beta2 = butterBandpass(filter_asr_signal, 15, 18, 1000)
filter_signal_beta3 = butterBandpass(filter_asr_signal, 18, 25, 1000)
for jj in range (0,20):
    for j in range (0,20):
        ch1_delta=filter_signal_delta[jj,:]
        ch2_delta=filter_signal_delta[j,:] 
        p_delta_1=np.log(np.mean(np.abs(ch1_delta**2)))
        p_delta_2=np.log(np.mean(np.abs(ch2_delta**2)))
        Amp_assym_delta[jj,j]=p_delta_1-p_delta_2
        
        ch1_theta=filter_signal_theta[jj,:]
        ch2_theta=filter_signal_theta[j,:] 
        p_theta_1=np.log(np.mean(np.abs(ch1_theta**2)))
        p_theta_2=np.log(np.mean(np.abs(ch2_theta**2)))
        Amp_assym_theta[jj,j]=p_theta_1-p_theta_2
        
        ch1_alpha=filter_signal_alpha[jj,:]
        ch2_alpha=filter_signal_alpha[j,:] 
        p_alpha_1=np.log(np.mean(np.abs(ch1_alpha**2)))
        p_alpha_2=np.log(np.mean(np.abs(ch2_alpha**2)))
        Amp_assym_alpha[jj,j]=p_alpha_1-p_alpha_2
        
        ch1_beta=filter_signal_beta[jj,:]
        ch2_beta=filter_signal_beta[j,:] 
        p_beta_1=np.log(np.mean(np.abs(ch1_beta**2)))
        p_beta_2=np.log(np.mean(np.abs(ch2_beta**2)))
        Amp_assym_beta[jj,j]=p_beta_1-p_beta_2
        
        ch1_h_beta=filter_signal_h_beta[jj,:]
        ch2_h_beta=filter_signal_h_beta[j,:] 
        p_h_beta_1=np.log(np.mean(np.abs(ch1_h_beta**2)))
        p_h_beta_2=np.log(np.mean(np.abs(ch2_h_beta**2)))
        Amp_assym_h_beta[jj,j]=p_h_beta_1-p_h_beta_2
        
        ch1_beta_1=filter_signal_beta1[jj,:]
        ch2_beta_1=filter_signal_beta1[j,:] 
        p_beta_1_1=np.log(np.mean(np.abs(ch1_beta_1**2)))
        p_beta_1_2=np.log(np.mean(np.abs(ch2_beta_1**2)))
        Amp_assym_beta1[jj,j]=p_beta_1_1-p_beta_1_2
        
        ch1_beta_2=filter_signal_beta2[jj,:]
        ch2_beta_2=filter_signal_beta2[j,:] 
        p_beta_2_1=np.log(np.mean(np.abs(ch1_beta_2**2)))
        p_beta_2_2=np.log(np.mean(np.abs(ch2_beta_2**2)))
        Amp_assym_beta2[jj,j]=p_beta_2_1-p_beta_2_2
        
        ch1_beta_3=filter_signal_beta3[jj,:]
        ch2_beta_3=filter_signal_beta3[j,:] 
        p_beta_3_1=np.log(np.mean(np.abs(ch1_beta_3**2)))
        p_beta_3_2=np.log(np.mean(np.abs(ch2_beta_3**2)))
        Amp_assym_beta3[jj,j]=p_beta_3_1-p_beta_3_2

# coherence

In [5]:
coherence1=np.zeros((21,21))
c_delta=np.zeros((4,21,21))
c_theta=np.zeros((4,21,21))
c_alpha=np.zeros((5,21,21))
c_beta=np.zeros((17,21,21))
c_gamma=np.zeros((15,21,21))
for jj in range (0,20):
    for j in range (0,20):
        ch_m=filter_asr_signal[jj,:]
        ch_n=filter_asr_signal[j,:]
        power_f=np.log(np.mean(np.abs(ch_m**2)))
        power_ff=np.log(np.mean(np.abs(ch_n**2)))
        fxx,cxx=ss.coherence(ch_m, ch_n, sampling_rate=1000, window='hann', nperseg=1000, noverlap=None, nfft=None, detrend='constant', axis=-1)
        c_delta[:,jj,j]=cxx[1:5]
        c_theta[:,jj,j]=cxx[5:9]
        c_alpha[:,jj,j]=cxx[9:13]
        c_beta[:,jj,j]=cxx[13:26]
        c_h_beta[:,jj,j]=cxx[26:31]
        c_beta1[:,jj,j]=cxx[13:16]
        c_beta2[:,jj,j]=cxx[16:19]
        c_beta3[:,jj,j]=cxx[19:26]

# phase lag

In [1]:
phase_diff=np.zeros((21,21))
for jj in range (0,20):
    for j in range (0,20):
        ch1_delta=filter_signal_delta[jj,:]
        ch2_delta=filter_signal_delta[j,:] 
        x1_delta= hilbert(ch1_delta)
        x2_delta = hilbert(ch2_delta)
        c_delta = np.inner( x1_delta, np.conj(x2_delta) )/np.sqrt( np.inner(x1_delta,np.conj(x1_delta)) * np.inner(x2_delta,np.conj(x2_delta)))
        phase_diff_delta[jj,j]= np.angle(c_delata)
       
        ch1_theta=filter_signal_theta[jj,:]
        ch2_theta=filter_signal_theta[j,:] 
        x1_theta= hilbert(ch1_theta)
        x2_theta = hilbert(ch2_theta)
        c_theta = np.inner( x1_theta, np.conj(x2_theta) )/np.sqrt( np.inner(x1_theta,np.conj(x1_theta)) * np.inner(x2_theta,np.conj(x2_theta)))
        phase_diff_theta[jj,j]= np.angle(c_theta)
       
        ch1_alpha=filter_signal_alpha[jj,:]
        ch2_alpha=filter_signal_alpha[j,:] 
        x1_alpha= hilbert(ch1_alpha)
        x2_alpha = hilbert(ch2_alpha)
        c_alpha = np.inner( x1_alpha, np.conj(x2_alpha) )/np.sqrt( np.inner(x1_alpha,np.conj(x1_alpha)) * np.inner(x2_alpha,np.conj(x2_alpha)))
        phase_diff_alpha[jj,j]= np.angle(c_alpha)
        
        ch1_beta=filter_signal_beta[jj,:]
        ch2_beta=filter_signal_beta[j,:] 
        x1_beta= hilbert(ch1_beta)
        x2_beta = hilbert(ch2_beta)
        c_beta= np.inner( x1_beta, np.conj(x2_beta) )/np.sqrt( np.inner(x1_beta,np.conj(x1_beta)) * np.inner(x2_beta,np.conj(x2_beta)))
        phase_diff_beta[jj,j]= np.angle(c_beta)
       
       
        ch1_h_beta=filter_signal_h_beta[jj,:]
        ch2_h_beta=filter_signal_h_beta[j,:] 
        x1_h_beta= hilbert(ch1_h_beta)
        x2_h_beta = hilbert(ch2_h_beta)
        c_h_beta = np.inner( x1_h_beta, np.conj(x2_h_beta) )/np.sqrt( np.inner(x1_h_beta,np.conj(x1_h_beta)) * np.inner(x2_h_beta,np.conj(x2_h_beta)))
        phase_diff_h_beta[jj,j]= np.angle(c_h_beta)
        
        ch1_beta1=filter_signal_beta1[jj,:]
        ch2_beta1=filter_signal_beta1[j,:] 
        x1_beta1= hilbert(ch1_beta1)
        x2_beta1 = hilbert(ch2_beta1)
        c_beta1= np.inner( x1_beta1, np.conj(x2_beta1) )/np.sqrt( np.inner(x1_beta1,np.conj(x1_beta1)) * np.inner(x2_beta1,np.conj(x2_beta1)))
        phase_diff_beta1[jj,j]= np.angle(c_beta1)
        
        ch1_beta2=filter_signal_beta2[jj,:]
        ch2_beta2=filter_signal_beta2[j,:] 
        x1_beta2= hilbert(ch1_beta2)
        x2_beta2 = hilbert(ch2_beta2)
        c_beta2 = np.inner( x1_beta2, np.conj(x2_beta2) )/np.sqrt( np.inner(x1_beta2,np.conj(x1_beta2)) * np.inner(x2_beta2,np.conj(x2_beta2)))
        phase_diff_beta2[jj,j]= np.angle(c_beta2)
       
        ch1_beta3=filter_signal_beta3[jj,:]
        ch2_beta3=filter_signal_beta3[j,:] 
        x1_beta3= hilbert(ch1_beta3)
        x2_beta3 = hilbert(ch2_beta3)
        c_beta3 = np.inner( x1_beta3, np.conj(x2_beta3) )/np.sqrt( np.inner(x1_beta3,np.conj(x1_beta3)) * np.inner(x2_beta3,np.conj(x2_beta3)))
        phase_diff_beta3[jj,j]= np.angle(c_beta3)
       

NameError: name 'np' is not defined

# DFA

In [ ]:
dfas=np.empty(21)
for i in range (21):
    if __name__=='__main__':
        x_delta = np.abs(ss.hilbert(filter_signal_delta[i]))
        scales, fluct, alpha_factor_delta = dfa(x_delta, show=1)
        dfas_delta[i]=alpha_factor_delta
        
         x_theta = np.abs(ss.hilbert(filter_signal_theta[i]))
        scales, fluct, alpha_factor_theta= dfa(x_theta, show=1)
        dfas_theta[i]=alpha_factor_theta
        
         x_alpha= np.abs(ss.hilbert(filter_signal_alpha[i]))
        scales, fluct, alpha_factor_alpha = dfa(x_alpha, show=1)
        dfas_alpha[i]=alpha_factor_alpha
        
         x_beta = np.abs(ss.hilbert(filter_signal_beta[i]))
        scales, fluct, alpha_factor_beta = dfa(x_beta, show=1)
        dfas_beta[i]=alpha_factor_beta
        
         x_h_beta = np.abs(ss.hilbert(filter_signal_h_beta[i]))
        scales, fluct, alpha_factor_h_beta = dfa(x_h_beta, show=1)
        dfas_h_beta[i]=alpha_factor_h_beta
        
         x_beta1 = np.abs(ss.hilbert(filter_signal_beta1[i]))
        scales, fluct, alpha_factor_beta1 = dfa(x_beta1, show=1)
        dfas_beta1[i]=alpha_factor_beta1
        
         x_beta2 = np.abs(ss.hilbert(filter_signal_beta2[i]))
        scales, fluct, alpha_factor_beta2 = dfa(x_beta2, show=1)
        dfas_beta2[i]=alpha_factor_bet2
        
         x_beta3 = np.abs(ss.hilbert(filter_signal_beta3[i]))
        scales, fluct, alpha_factor_beta3 = dfa(x_beta3, show=1)
        dfas_beta3[i]=alpha_factor_beta3
        